In [1]:
import numpy as np

In [2]:
import crop
import soil

### 2DMAIZSIM.for

In [3]:
soil.datafilenames.runfile[:] = 'test.dat'

In [4]:
soil.initialize()
soil.get_grid_and_boundary()
soil.synchronizer()

In [5]:
if sum([bool(f) for f in [soil.time_public.hourlyweather, soil.time_public.dailyweather]]) != 1:
    print "error in weather file type"

In [6]:
if soil.time_public.hourlyweather:
    soil.setsurfaceh()
if soil.time_public.dailyweather:
    soil.setsurfaced()

In [7]:
soil.settdb()
soil.autoirrigate()
soil.mngm()

In [8]:
soil.datafilenames.starter = 1.0

### Crop.cpp

#### Common Blocks

In [9]:
shootr = soil.shootr
weather = soil.weath
grid_public = soil.grid_public
node_public = soil.nodal_public
#ele_public = soil.elem_public
#bound_public = soil.bound_public
time_public = soil.time_public
module_public = soil.module_public
file_public = soil.datafilenames

#### Controller Initiation

Crop module initiaition is done only once when first ran.

In [10]:
time_public.linput

array(1, dtype=int32)

Parse the file names from the FORTRAN strings passed from 2dsoil.

In [11]:
varFile = file_public.varietyfile.tostring().strip()
graphicFile = file_public.plantgraphics.tostring().strip()
leafFile = file_public.leafgraphics.tostring().strip()

Set up parameters from initials file whose reading is moved to soil model (Init.for).

In [12]:
initInfo = crop.TInitInfo()

In [13]:
initInfo.plantDensity = shootr.poparea.item()
initInfo.latitude = weather.latude.item()
initInfo.longitude = weather.longitude.item()
initInfo.altitude = weather.altitude.item()
initInfo.year = time_public.year.item()
initInfo.sowingDay = time_public.sowingday.item()
initInfo.beginDay = time_public.beginday.item()
initInfo.endDay = time_public.endday.item()
initInfo.timeStep = time_public.timestep.item()

In [14]:
shootr.lcai = 0
shootr.lareat = 0
shootr.height = 0
# change for debugging purposes
shootr.convr = 1 # was 0.1 or 0.38 should be 1.0 as dry matter is used in all cases
shootr.awups = 0 # initialize AWUPS, AWUPS_old and LeafWP in 2DSOIL
shootr.leafwp = -0.5
shootr.pcrs = 0
shootr.et_demand = 0
shootr.hourlycarboused = 0 # it is also zero'd upon initialization in 2dsoil

In [15]:
# period should be in days, input in minutes
period = time_public.timestep / 60. / 24.

In [16]:
popSlab = shootr.poprow / 100. * shootr.eomult

These two lines show the relationships among some of the space variables.

In [17]:
#popSlab = shootr.poprow / 100. * shootr.rowsp * shootr.eomult
#plantDensity = shootr.poprow * 100. / shootr.rowsp

A new plant model object is created and initialized (calls initialize function).

In [18]:
pSC = crop.CController(varFile, graphicFile, leafFile, initInfo)

#### Common Objects

In [19]:
plant = pSC.getPlant()
develop = plant.get_develop()
initInfo = pSC.getInitInfo()

#### Reset uptakes

Initialize nitrogen uptake with what is already in the plant.

In [20]:
# nitrogen uptake value from 2dsoil accumulated between time steps mg/plant
#SK 8/20/10: This is curious but OK
nitrogenUptake = plant.get_N() * popSlab

In [21]:
# hourly water uptake from 2dsoil
waterUptake = 0.

In [22]:
shootr.ndemanderror = 0
shootr.cumulativendemanderror = 0

In [23]:
time_public.runflag = 1

In [24]:
module_public.nummod += 1

In [25]:
# module number
modNum = module_public.nummod

In [26]:
time_public.tnext[modNum - 1] = pSC.getSowingDay()

#### Some variables that need to be initialized (from Crop.h)

In [27]:
cumulativeNUptakeError = 0.

In [28]:
SLNmin = 0.5

In [29]:
predawnLWP = -0.05

In [30]:
old_shoot_weightPerM2 = 0.

In [31]:
nitrogenUptakeOld = 0.

In [32]:
cumulativeNitrogenDemand = 0. # grams plant-1

In [33]:
d = 0.075 # d: shape coefficient in the logistic function to simulate cumulative N uptake (Equation 9 in Lindquist et al. 2007)
q_n = 0.032 # q_n: the maximum ratio of daily N uptake to measured daily growth rate (g N g-1) (Lindquist et al., 2007)
a = 4.10 # maximum nitrogen concentration for C4 species: 4.1% (Lindquist et al. 2007)
b = 0.5 # shape coefficient in calculation of nitrogen concentration in relation to up-ground biomass (equation 4 Lindquist et al, 2007) YY

#### Running the crop module step by step

In [34]:
if module_public.nshoot > 0:
    waterUptake += shootr.awups * time_public.step
    nitrogenUptake += shootr.sincrsink / 1.0e6

Note that SIncrSink has been multiplied by time step in the solute uptake routing the 1000 scales from ug to mg.

In [35]:
abs(time_public.time - time_public.tnext[modNum - 1]) < abs(0.001 * time_public.step)

False

#### Only if the codition is met

If the sowing date has come and there is not plant, let the program know so other calculations are not done.

In [36]:
if module_public.nshoot == 0 and abs(time_public.time - pSC.getSowingDay()) < 0.001:
    module_public.nshoot = 1

Calculate error for demand and actual uptake, if negative, demand is greater then uptake.

In [37]:
currentNUptakeError = nitrogenUptake / popSlab - plant.get_CumulativeNitrogenDemand()

In [38]:
cumulativeNUptakeError += currentNUptakeError

#### TWeather

In [39]:
wthr = crop.TWeather()

In [40]:
i = time_public.itime - 1
wthr.HourlyOutput = time_public.hourlyoutput.item()
wthr.DailyOutput = time_public.dailyoutput.item()
wthr.jday = weather.jday.item()
wthr.time = time_public.time - wthr.jday
wthr.CO2 = weather.co2.item()
wthr.airT = weather.tair[i].item()
wthr.PFD = weather.par[i] * 4.6 # conversion from PAR in Wm-2 to umol s-1 m-2
wthr.solRad = weather.wattsm[i].item() # conversion from Wm-2 to J m-2 in one hour Total Radiation incident at soil surface
Es = 0.611 * np.exp(17.502 * wthr.airT / (240.97 + wthr.airT)) # saturated vapor pressure at airT
wthr.RH = (1 - weather.vpd[i] / Es) * 100.
wthr.rain = weather.rint[i].item()
wthr.wind = weather.wind * (1000. / 3600.) # conversion from km hr-1 to m s-1
wthr.dayLength = weather.daylng.item()
# since LeafWP in 2dsoil is in bar but in maizesim is in MPa, so, have to divide it by 10 to convert it into MPa before passing the value to Maizesim 1 bar=10kPa
wthr.LeafWP = shootr.psil_ / 10. # and leaf water potential information into MAIZESIM Yang 8/15/06
wthr.pcrl = shootr.pcrl / popSlab / 24.
wthr.pcrq = shootr.pcrq / popSlab / 24.

If time is 5 am, then pass the leaf water potential (the predawn leaf water potential) from SHOOTR to the wthr object. YY

In [41]:
if abs(wthr.time - 0.2083) < 0.0001:
    # Here LeafWP is in bar.
    # Since the LWPeffect in leaf.cpp uses leaf water potential in bar,
    # so here PredawnLWP is in bar, instead of being scaled to MPa. YY
    predawnLWP = shootr.psil_.item() # SHOOTR->LeafWP

Pass actual carbohydrate amount used in 2dsoil back to the plant.

In [42]:
#ToDo - make pcrs a new variable (ActualRootCarboUsed) and make it a member of plant.
#dt here I changed this temporarily for debugging
# don't need to divide by 24 since the value has been integrated over an hour
wthr.pcrs = shootr.hourlycarboused / popSlab # original
shootr.hourlycarboused = 0.
# dividing it by PopSlab converts it to g/day/plant
#ToDo: need to document this better, what is pcrs being used for.

SHOOTR->PCRS in 2dsoil is the actual rate of carbon supplied to roots in a soil slab, it is in g/day.
 - dividing it by (SHOOTR->Rowsp*1)/10000 converts it to g/day/m^2
 - further dividing it by weather->daylng converts it to g/hour/m^2
 - then dividing it by plant density, converts it to g/hour/plant, which is the unit of the wthr.pcrs in maizesim. Yang. 10/27/06

Pass through nitrogen uptake (total mg per slab in the one hour) from 2DSOIL.

In [43]:
wthr.TotalRootWeight = shootr.totalrootweight / popSlab
wthr.MaxRootDepth = shootr.maxrootdepth.item()
# Available water is cm per profile - should be divided by PopSlab
wthr.ThetaAvail = node_public.thetaavail / popSlab

In [44]:
if nitrogenUptake > 0:
    nuptake = nitrogenUptake / popSlab
    leafloss = plant.get_droppedLfArea()
    nloss = leafloss * SLNmin
    # SLNmin: base Specific leaf nitrogen content; for now assume it's 0.5 YY
    
    #SK 8/20/10: Here seems to be the only place where totalN of the plant is set.
    # NitrogenUptake is initiated from get_N at the begining of the timestep so OK. 
    plant.set_N(nuptake)
    # Units are converted from g slab-1 to g plant -1 YY
    # need to look at loss of N in the dropped leaf (plant N goes negative?)				                                                         
    #pSC.getPlant().set_N(nuptake - nloss)

In [45]:
if shootr.lai == 0:
    wthr.ET_supply = 0.
else:
    # Note water uptake has been summed over the past hour so it is an hourly amount
    # into MAIZESIM Yang 8/15/06, dt 4/24/2011
    wthr.ET_supply = waterUptake / (shootr.eomult * shootr.poprow) * 100.
    #dt 4-24-2011 I replaced SHOOTR->AWUPS with WaterUptake. AWUPS is an instantaneous value.
    
    # for debugging
    ET_diff = wthr.ET_supply * 24 - shootr.et_demand

ET_Supply is the actual amount of water that can be taken from the soil slab ( derived from AWUPS, g day-1 slab-1). To compare this variable with the et rate in maizesim it has to be converted into grams water per plant. To do this multiply by EOMULT to double slab width if plant is at the edge. Then multiply by 100/PopRow to get area inhabited by the plant. This provides a per plant estimate from area.

First find top of grid.

In [46]:
y = grid_public.y[:grid_public.numnp]
maxY = y.max()
lowerBoundary = maxY - 5.

Now find average temperature in layer between surface and lower boundary.

In [47]:
ylb = (y >= lowerBoundary)
soilT = node_public.tmpr[ylb].sum()
count = ylb.sum()

In [48]:
wthr.soilT = soilT / (count*1.)

The model code to simulate growth ect begins here when the plant object is called.

In [49]:
#TODO add some error catching code here
ier = pSC.getErrStatus()
if ier == 0:
    ier = pSC.run(wthr, predawnLWP)
    # Pass both weather and leaf water potential into the "run" function
    # of the controller pSC YY
    # need to get rid of other run module (with only wthr) done?
    # need to add PredawnLWP to wthr structure

Assumes that germination takes place about halfway through the sowing date.

In [50]:
if 0.49 <= wthr.time <= 0.51:
    if not develop.Germinated():
        print wthr.jday

ActualCarboIncrement is calculated from "assimilate", which in turn is calculated from photosynthsis_net in plant; the unit of assimilate then is in g/plant/hour, thus, at this point, pcrl has unit g/plant/hour. Multiply by 24 to get g plant-1 day-1; multiply by popslab to get g Carbo slab-1 day-1.

In [51]:
if develop.Emerged():
# pass appropriate data to 2DSOIL file structures    
    #dt 03/14/2011- I added a pool of carbo to hold leftover carbon from root growth, here it is implemented - see also plant
    # This holds any carbon not used for root growth in the previous time step
    pool = plant.get_C_pool_root()
    root = plant.get_rootPart()
    shoot = plant.get_shootPart()
    
    # this assures the pool is only used at night
    # minimizes complexity when pcrq has a value
    # since we have to add leftover carbo from pcrq to the shoot
    if pool > 0 and root < 0.00001:
        shootr.pcrl = (root + pool) * 24*popSlab
        plant.set_C_pool_root(0.)
    else:
        shootr.pcrl = root * 24*popSlab
    
    if develop.GrainFillBegan():
        shootr.pcrq = (root + 0.75*shoot) * 24*popSlab
    else:
        shootr.pcrq = (root + shoot) * 24*popSlab
    
    #DT 09/19/14 under strong water stress mid season too much carbon is allocated to the roots,
    #we try to limit it here.
    #SHOOTR->PCRQ=SHOOTR->PCRL; //for debugging now remove later
    #dt 03/2011 added these two for debugging now
    #need to calculate mass balcance of carbo sent to root
    #can drop them later
    wthr.pcrl = shootr.pcrl / popSlab/24.
    wthr.pcrq = shootr.pcrq / popSlab/24.
    
    shootr.lcai = plant.calcGreenLeafArea() * initInfo.plantDensity
    shootr.cover = 1 - np.exp(-0.79*shootr.lcai)
    shootr.shade = shootr.cover * shootr.rowsp
    shootr.height = min(shootr.shade, shootr.rowsp)
    shootr.et_demand = plant.get_ET() * 24 # pass ET demand from shoot to root. Yang
    # In GasExchange, the unit of ET is mmol m-2(leaf) sec-1
    # need to convert to grams plant-1
    # Here, multiplying ET by 0.018 and 3600*24 converts it to g m-2(ground) day-1
    # dividing it by plantdensity converts it to g plant-1 day-1
    shootr.lai = shootr.lcai
    shoot_weightPerM2 = plant.get_shootMass() * initInfo.plantDensity # Calculate total shoot mass per meter aquared YY
    massIncrease = shoot_weightPerM2 - old_shoot_weightPerM2 # Calculated increase in above-ground biomass per m2 YY
    
    # The biomass  returned by getPlant()->get_shootMass() is the weight of each single plant (g/plant),
    # to convert it into (g/m-2), it has to be 
    # multiplied by pSC->getIniInfo().plantDensity
    
    # Perhaps it's good idea to merge all plant N business into plant.cpp where C allocation is taken care of.
    # For now I'm not modifying any codes here. TODO - dt 03/15/2011 Much of this nitrogen stuff can be cleaned up as some of
    # Yang's original code is no longer used (i.e., U_N etc).
    
    nitrogenRatio = a / 100.
    if shoot_weightPerM2 >= 100:
        # Calcualte above ground potential N concentration
        #nitrogenRatio *= np.sqrt(shoot_weightPerM2)
        nitrogenRatio *= pow(shoot_weightPerM2, 1 - b)
    plant.set_NitrogenRatio(nitrogenRatio / 10.)
    
    # U_N maximum observed N uptake rate (g N m-2 ground d-1) (Lindquist et al, 2007) YY
    # The unit of U_N is g N m-2 ground d-1
    #d = 0.075 # d: shape coefficient in the logistic function to simulate cumulative N uptake (Equation 9 in Lindquist et al. 2007)
    U_N = 0.359 * d / 4.
    
    # U_M maximum uptake rate as limited by maximum N fraction per unit (Equation 2 in Lindquist et al., 2007)
    # unit of U_M is also g N m-2 ground d-1; however, the unit of massIncrease is g m-2/step (here one hour). 
    # Here in the simulation, the default length of one step is an hour; so, we have to scale it up to one
    # day by multiplying it by 24
    #q_n = 0.032 # q_n: the maximum ratio of daily N uptake to measured daily growth rate (g N g-1) (Lindquist et al., 2007)
    U_M = q_n * massIncrease * 24

    # unit of U_P is also g N m-2 ground d-1; however, the unit of massIncrease is g/step. Here
    # in the simulation, the default length of one step is an hour; so, we have to scale it up to one
    # day by multiplying it by 24
    
    # U_P potential rate of N accumulation (g N m-2 ground d-1) (Lindquist et al. 2007)
    # if shoot weight<100 (g m-2) then U_P is calculated this way
    U_P = (a / 100.) * massIncrease * 24
    # otherwise, it is calculated like this (Equation 6, Lindquist et al., 2007) YY
    if shoot_weightPerM2 >= 100:
        U_P *= 10 * (1 - b) * pow(shoot_weightPerM2, -b)

    # U_D U uptake rate (g N m-2 d-1) as limited by the difference between potential and actual amount of N 
    # in existing biomass, equation 3 in Lindquist et al. 2007)
    # the returned value from get_N() is in g N/plant. It has to be converted to g/m-2 ground
    # that's why the actual n content is mulitpled by pSC->getIniInfo().plantDensity/(100*100) YY
    U_D = a * 10 / 100. * pow(shoot_weightPerM2, -b) - plant.get_N() * initInfo.plantDensity
    
    # set up account of N here
    # first hourly
    # Actual and needed N uptake in the last hour per plant per day
    
    # houly rate per day
    hourlyActualNFromSoil = (nitrogenUptake - nitrogenUptakeOld) / popSlab
    plant.set_HourlyNitrogenSoilUptake(hourlyActualNFromSoil)
    
    # Determine the nitrogen demand (equation 1 Lindquist et al. 2007) in grams plant-1
    hourlyNitrogenDemand = max(U_P, 0) / initInfo.plantDensity / 24.
    plant.set_HourlyNitrogenDemand(hourlyNitrogenDemand)
    
    # now do cumulative amounts
    cumulativeNitrogenDemand += hourlyNitrogenDemand # grams plant-1 day-1
    plant.set_CumulativeNitrogenDemand(cumulativeNitrogenDemand)
    plant.set_CumulativeNitrogenSoilUptake(nitrogenUptake / popSlab)
    
    # Pass the nitrogen demand into 2dsoil YY
    # Units are ug slab-1
    #oldNDemand = shootr.nitrodemand / popSlab / 1e6 / 24.
    shootr.nitrodemand = hourlyNitrogenDemand * popSlab * 1e6 * 24
    
    # Save the value of the above_ground biomass of this time-step
    old_shoot_weightPerM2 = shoot_weightPerM2
    
    # save the cumulative N uptake from this time step
    nitrogenUptakeOld = nitrogenUptake
    
    shootr.ndemanderror = currentNUptakeError
    shootr.cumulativendemanderror = cumulativeNUptakeError

In [52]:
if develop.Dead():
#if pSC.getLastDayOfSim() <= pSC.getTime().get_day_of_year():
    print "Completing crop simulation..."
    
    # tell 2dsoil that crops harvested
    module_public.nshoot = 0
    
    # set the next time so the model
    time_public.tnext[modNum - 1] = 1e12
    
    # if matured points to nothing
    pSC = None
    
    time_public.runflag = 0
else:
    time_public.tnext[modNum - 1] = time_public.time + period
    waterUptake = 0.
    #nitrogenUptake = 0.

### 2DMAIZSIM.for (back)

In [53]:
soil.carbon_partitioning()
soil.rootgrow()
soil.wateruptake()
soil.soluteuptake()
soil.gasuptake()

In [54]:
soil.watermover()
soil.solutemover()
soil.heatmover()
soil.gasmover()
soil.soilnitrogen()
soil.macrochem()
#soil.massbl()

In [55]:
if time_public.outputsoilyes > 0:
    soil.output()

In [56]:
#GOTO 1